In [1]:
!git clone https://github.com/chrishendra93/MI_Workshop


Cloning into 'MI_Workshop'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 79 (delta 33), reused 53 (delta 19), pack-reused 0
Unpacking objects: 100% (79/79), done.


In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import torch
from tqdm.notebook import tqdm

In [2]:
# Setting random seed so that everything is replicable

np.random.seed(0)
torch.random.manual_seed(0)

In [3]:
root_dir = "/content/MI_Workshop/mnist_clr"
data_dir = "/content/MI_Workshop/lab_2"

mnist_columns = ["label"] + ["features_{}".format(i) for i in range(28 ** 2)]
mnist_train = pd.read_csv("./sample_data/mnist_train_small.csv", names=mnist_columns)
mnist_test = pd.read_csv("./sample_data/mnist_test.csv", names=mnist_columns)
mnist_arr_train = mnist_train[["features_{}".format(i) for i in range(28 ** 2)]].values
mnist_arr_test = mnist_test[["features_{}".format(i) for i in range(28 ** 2)]].values

X_train = np.load(os.path.join(root_dir, "train_features.npy"))
X_test = np.load(os.path.join(root_dir, "test_features.npy"))
y_train = np.load(os.path.join(root_dir, "train_labels.npy"))
y_test = np.load(os.path.join(root_dir, "test_labels.npy"))

print(np.all(y_train == mnist_train["label"].values))
print(np.all(y_test == mnist_test["label"].values))

train_bags, train_labels = np.load(os.path.join(data_dir, "train_bags.npy"), allow_pickle=True), np.load(os.path.join(data_dir, "train_labels.npy"),allow_pickle=True)
test_bags, test_labels = np.load(os.path.join(data_dir, "test_bags.npy"), allow_pickle=True), np.load(os.path.join(data_dir, "test_labels.npy"), allow_pickle=True)


True
True


In [9]:
# Visualize distribution of labels in train and test sets

In [12]:
# Visualize distribution of instances in train and test sets



In [4]:
def visualize_bags(mnist_arr, bags, bag_labels, n_bags):
  pos_bags = np.argwhere(bag_labels == 1).flatten()
  neg_bags = np.argwhere(bag_labels == 0).flatten()
  n_pos = (n_bags // 2 )
  n_neg = n_bags - n_pos
  sampled_indices = np.concatenate([np.random.choice(pos_bags, n_pos, replace=False), np.random.choice(neg_bags, n_neg, replace=False)])
  sampled_bags = bags[sampled_indices]
  sampled_bag_labels = bag_labels[sampled_indices] 
  max_instances_num = np.max([len(bag) for bag in sampled_bags])
  _, axes = plt.subplots(len(sampled_bags), max_instances_num,
                         figsize=(10 * len(sampled_bags), 
                                  10 * max_instances_num))
  for idx, bag, bag_label in zip(np.arange(n_bags), sampled_bags, sampled_bag_labels):
    ax = axes[idx, :]
    instance_size = None
    for i in range(max_instances_num):
      if i >= len(bag):
        ax[i].imshow(np.zeros(instance_size)) # Pad with empty images if bag has fewer instances than max
      else:
        img = mnist_arr[bag[i]]
        if instance_size is None:
          w = int(np.sqrt(len(img)))
          instance_size = (w, w)
        ax[i].imshow(img.reshape(instance_size))
      if i == max_instances_num // 2:
        ax[i].set_title('Bag Label: {}'.format(bag_label), fontsize=50)
  plt.subplots_adjust(bottom=0.1, top=0.3, hspace=0.2)


In [14]:
# Visualize the instances within bags


In [5]:
from scipy.stats import kurtosis
from functools import partial
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def fetch_instances(mnist_features, indices):
  return mnist_features[indices]

def extract_summary_stats(mnist_features, bags, summary_funcs=[np.mean]):
  summary_stats = []
  for summary_func in summary_funcs:
    summary_func = partial(summary_func, axis=0)
    summary_stat = np.array([summary_func(mnist_features[bag]) for bag in bags]).reshape(-1, 64)
    summary_stats.append(summary_stat)
  return np.concatenate(summary_stats, axis=1)

In [26]:
# Extract summary statistics using mean, variance and kurtosis just as before and visualize the pca plots 


In [25]:
# fig, ax = plt.subplots(1, 1, figsize=(20, 20))
# to_plot_train = pd.DataFrame({'x': X_train_pca[:, 0].flatten(), 
#                               'y': X_train_pca[:, 1].flatten(),
#                               'labels': train_labels})
# to_plot_train["labels"] = to_plot_train["labels"].astype('category')
# sns.scatterplot(x='x', y='y', data=to_plot_train, hue='labels', ax=ax)

In [24]:
# fig, ax = plt.subplots(1, 1, figsize=(20, 20))
# to_plot_test = pd.DataFrame({'x': X_test_pca[:, 0].flatten(), 
#                               'y': X_test_pca[:, 1].flatten(),
#                               'labels': test_labels})
# to_plot_test["labels"] = to_plot_test["labels"].astype('category')
# sns.scatterplot(x='x', y='y', data=to_plot_test, hue='labels', ax=ax)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_curve, precision_recall_curve, auc
from scipy.stats import mode

In [7]:
def get_roc_auc(y_true, y_pred):
    fpr, tpr, _  = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)
    return roc_auc


def get_pr_auc(y_true, y_pred):
    precision, recall, _ = precision_recall_curve(y_true, y_pred, pos_label=1)
    pr_auc = auc(recall, precision)
    return pr_auc


def get_accuracy(y_true, y_pred):
    return balanced_accuracy_score(y_true, y_pred)

In [30]:
# summary_statistics_list = {'mean': [np.mean], 'var': [np.var], 'kurtosis': [kurtosis],
#                            'mean_var': [np.mean, np.var], 'mean_kurtosis': [np.mean, kurtosis], 'var_kurtosis': [np.var, kurtosis],
#                            'mean_var_kurtosis': [np.mean, np.var, kurtosis]}

# df = []
# for summary_statistics, stats_func in summary_statistics_list.items():

#   X_train_stats = extract_summary_stats(X_train, train_bags, stats_func)
#   X_test_stats = extract_summary_stats(X_test, test_bags, stats_func)
#   model = LogisticRegression(max_iter=10000).fit(X_train_stats, train_labels)
#   y_test_pred = model.predict(X_test_stats)
#   balanced_accuracy = balanced_accuracy_score(test_labels, y_test_pred)
#   roc_auc = get_roc_auc(test_labels, y_test_pred)
#   pr_auc = get_pr_auc(test_labels, y_test_pred)
#   df += [(summary_statistics, roc_auc, pr_auc, balanced_accuracy)]

# df = pd.DataFrame(df, columns=['Model', 'ROC AUC', 'PR AUC', 'Balanced Accuracy']).sort_values("Balanced Accuracy", ascending=False)
# df

In [8]:
from sklearn.metrics import pairwise_distances

def get_hausdorff_distance(X_train, X_test, train_bags, test_bags, mode='max', quantile=0.7):
  pairwise_distance = pairwise_distances(X_test, X_train)
  all_bag_distances = []
  
  for test_bag in tqdm(test_bags, total=len(test_bags)):
    test_bag_instances = pairwise_distance[test_bag]
    bag_distances = []

    for train_bag in train_bags:
      if mode == 'max':
        test_train_distance = np.max(np.min(test_bag_instances[:, train_bag], axis=1))
        train_test_distance = np.max(np.min(test_bag_instances[:, train_bag].transpose(), axis=1))
        bag_distances.append(max(test_train_distance, train_test_distance))
      
      elif mode =='avg':
        test_train_distance = np.min(test_bag_instances[:, train_bag], axis=1)
        train_test_distance = np.min(test_bag_instances[:, train_bag].transpose(), axis=1)
        bag_distances.append(np.mean(np.concatenate([train_test_distance, test_train_distance])))
      
      elif mode == 'min':
        test_train_distance = np.min(np.min(test_bag_instances[:, train_bag], axis=1))
        train_test_distance = np.min(np.min(test_bag_instances[:, train_bag].transpose(), axis=1))
        bag_distances.append(max(test_train_distance, train_test_distance))
      elif mode == 'quantile':
        test_train_distance = np.quantile(np.min(test_bag_instances[:, train_bag], axis=1), quantile)
        train_test_distance = np.quantile(np.min(test_bag_instances[:, train_bag].transpose(), axis=1), quantile)
        bag_distances.append(max(test_train_distance, train_test_distance))
    all_bag_distances.append(np.array(bag_distances).reshape(1, -1))
  all_bag_distances = np.concatenate(all_bag_distances, axis=0)
  return all_bag_distances


In [34]:
# bag_distances_max = get_hausdorff_distance(X_train, X_test, train_bags, test_bags, mode='max')
# bag_distances_avg = get_hausdorff_distance(X_train, X_test, train_bags, test_bags, mode='avg')
# bag_distances_min = get_hausdorff_distance(X_train, X_test, train_bags, test_bags, mode='min')

In [9]:
from scipy.stats import mode

def get_knn_pred(train_labels, test_labels, all_bag_distances, n_neighbors):
  return mode(train_labels[np.argsort(all_bag_distances, axis=1)[:, :n_neighbors]], axis=1)[0].flatten()


def get_citation_knn_pred_normalized(train_labels, test_labels, all_bag_distances, n_ref_neighbors=10, n_cit_neighbors=10):
  reference_nn = train_labels[np.argsort(all_bag_distances, axis=1)[:, :n_ref_neighbors]]
  num_pos_neighbors = np.sum(reference_nn, axis=1)
  num_neg_neighbors = n_ref_neighbors - num_pos_neighbors
  reference_nn_labels = np.concatenate([num_neg_neighbors.reshape(-1, 1),
                                        num_pos_neighbors.reshape(-1, 1)], axis=1)
  citer_nn_labels = get_citation_counts(train_labels, test_labels, all_bag_distances, n_cit_neighbors)
  return reference_nn_labels / np.sum(reference_nn_labels, axis=0) + citer_nn_labels / np.sum(citer_nn_labels, axis=0)

def get_citation_counts(train_labels, test_labels, all_bag_distances, n_neighbors):
  test_pos_neg_counts = np.zeros((len(test_labels), 2))
  nn = np.argsort(all_bag_distances.transpose(), axis=1)[:, :n_neighbors]
  for citer in range(len(nn)):
    citations = nn[citer]
    citer_label = np.array([0, 1]) if train_labels[citer] == 1 else np.array([1, 0])
    for citation in citations:
      test_pos_neg_counts[citation, :] += citer_label
  return test_pos_neg_counts


In [38]:
# n_neighbors_ref = 15
# n_neighbors_cit = 5

# df_knn = []

# for distance_mode, distance in zip(['max', 'avg', 'min'], 
#                           [bag_distances_max, bag_distances_avg, bag_distances_min]):
#   y_pred_test_knn = get_knn_pred(train_labels, test_labels, distance, n_neighbors_ref)
#   balanced_acc = balanced_accuracy_score(test_labels, y_pred_test_knn)
#   df_knn += [(distance_mode, balanced_acc)]
# df_knn = pd.DataFrame(df_knn, columns=['Distance Type', 'Balanced Accuracy']).sort_values("Balanced Accuracy", ascending=False)

# df_cknn = []

# for distance_mode, distance in zip(['max', 'avg', 'min'], 
#                           [bag_distances_max, bag_distances_avg, bag_distances_min]):
#   y_pred_test_knn = np.argmax(get_citation_knn_pred(train_labels, test_labels, distance, n_neighbors_ref, n_neighbors_cit), axis=1)
#   balanced_acc = balanced_accuracy_score(test_labels, y_pred_test_knn)
#   df_cknn += [(distance_mode, balanced_acc)]
# df_cknn = pd.DataFrame(df_cknn, columns=['Distance Type', 'Balanced Accuracy']).sort_values("Balanced Accuracy", ascending=False)

# df = df_knn.merge(df_cknn, on=["Distance Type"], suffixes=['_knn', '_cknn'])
# df

,Distance Type,Balanced Accuracy_knn,Balanced Accuracy_cknn
0,avg,0.792897,0.787266
1,max,0.708849,0.701752
2,min,0.613175,0.618387


Next we try instance based method, i.e, product rule and sum rule. First we train a classifier to tell whether an instance belongs to a positive or negative bag.

\\
Remember the product rule is given by:

$$p(+|X_i) \propto p(+)^{-(n_i - 1)} \prod_{i=1}^{n_i} p(+|x_{ij})p(x_{ij})$$
$$p(-|X_i) \propto p(-)^{-(n_i - 1)} \prod_{i=1}^{n_i} p(-|x_{ij})p(x_{ij})$$

So essentially we can compute

$$p(+|X_i) = p(+)^{-(n_i - 1)} \prod_{i=1}^{n_i} p(+|x_{ij})$$
$$p(-|X_i)p(-)^{-(n_i - 1)} \prod_{i=1}^{n_i} p(-|x_{ij})$$

and normalize the two so that they sum up to 1. Similarly we can approximate these two quantities using sum-rule and normalize the two quantities given below:

$$p(+|X_i) = (1-n_i)p(+) + \sum_{i=1}^{n_i} p(+|x_{ij})$$
$$p(-|X_i) = (1-n_i)p(-) + \sum_{i=1}^{n_i} p(-|x_{ij})$$


In [10]:
def product_rule(y_instance_proba, pos_prior, neg_prior):
  # This function works on one individual bag
  # y_instance proba is of the shape (N_i, 2)

  n_instances = len(y_instance_proba)
  p_neg = (neg_prior) ** (n_instances - 1) * np.product(y_instance_proba[:, 0])
  p_pos = (pos_prior) ** (n_instances - 1) * np.product(y_instance_proba[:, 1])

  return np.array([p_neg, p_pos]) / (p_neg + p_pos)

def sum_rule(y_instance_proba, pos_prior, neg_prior):
  # This function works on one individual bag
  # y_instance proba is of the shape (N_i, 2)

  n_instances = len(y_instance_proba)
  p_neg = ((neg_prior) ** (1 - n_instances)) * np.product(y_instance_proba[:, 0])
  p_pos = ((pos_prior) ** (1 - n_instances)) * np.product(y_instance_proba[:, 1])

  return np.array([p_neg, p_pos]) / (p_neg + p_pos)

def sum_rule(y_instance_proba, pos_prior, neg_prior):
  # This function works on one individual bag
  # y_instance proba is of the shape (N_i, 2)

  n_instances = len(y_instance_proba)
  p_neg = (1 - n_instances) * (neg_prior) + np.sum(y_instance_proba[:, 0])
  p_pos = (1 - n_instances) * (pos_prior) + np.sum(y_instance_proba[:, 1])

  return np.array([p_neg, p_pos]) / (p_neg + p_pos)

In [43]:
# Create training labels for instance-level classifier and train a Logistic Regression model to predict whether an instance belongs to a positive or negative bag
  

In [44]:
## Compare the results of the various rules


# df = []
# neg_prior, pos_prior = np.unique(train_labels, return_counts=True)[1] / len(train_labels)

# for mode, mi_func in zip(['max', 'mean', 'prod_rule', 'sum_rule'], [np.mean, np.max, product_rule, sum_rule]):
#   if mode in ('max', 'mean'):
#     y_test_pred_proba = np.concatenate([mi_func(y_test_instance_pred[bag], axis=0).reshape(-1, 2) for bag in test_bags])
#   else:
#     y_test_pred_proba = np.concatenate([mi_func(y_test_instance_pred[bag], pos_prior, neg_prior).reshape(-1, 2) for bag in test_bags])
#   y_test_pred = np.argmax(y_test_pred_proba, axis=1)
#   balanced_accuracy = balanced_accuracy_score(test_labels, y_test_pred)
#   roc_auc = get_roc_auc(test_labels, y_test_pred)
#   pr_auc = get_pr_auc(test_labels, y_test_pred)
#   df += [(mode, roc_auc, pr_auc, balanced_accuracy)]

# df = pd.DataFrame(df, columns=['Model', 'ROC AUC', 'PR AUC', 'Balanced Accuracy']).sort_values("Balanced Accuracy", ascending=False)
# df

In [11]:
import torch.nn.functional as F
import torch
from torch import nn
from torch.nn import BCELoss
from torch.optim import LBFGS, Adam
from torch.utils.data import Dataset, DataLoader

In [12]:
class LogisticRegressionMI(nn.Module):

  def __init__(self, n_dim, mode='max'):
    super(LogisticRegressionMI, self).__init__()
    if mode not in ('max', 'mean'):
      raise ValueError("Invalid mode {}, must be one of max or mean".format(mode))
    self.mode = mode
    self.encoder = nn.Linear(n_dim, 1)
    
  def forward(self, x, indices):
    x = self.encoder(x)
    x = torch.sigmoid(x)
    if self.mode == 'max':
      x = torch.stack([torch.max(x[idx]) for idx in indices])
    else:
      x = torch.stack([torch.mean(x[idx]) for idx in indices])
    return x
  
  def get_max_indices(self, x, indices):
    x = self.encoder(x)
    x = torch.sigmoid(x)
    pred, max_indices = [], []
    for idx in indices:
      max_idx = torch.argmax(x[idx])
      max_indices.append(idx[max_idx])
      pred.append(x[idx][max_idx])
    return torch.cat(pred), np.array(max_indices)


In [18]:
# logit_mi_max = LogisticRegressionMI(64, mode='max')
# criterion = BCELoss()
# optimizer = LBFGS(logit_mi_max.parameters(), lr=0.001, max_iter=1000)
# logit_mi_max.train()


# def closure():
#     optimizer.zero_grad()
#     output = logit_mi_max(torch.Tensor(X_train), train_bags)
#     loss = criterion(output, torch.Tensor(train_labels))
#     loss.backward()
#     return loss

# optimizer.step(closure) 

# logit_mi_max.eval()
# with torch.no_grad():
#   y_pred_logit_mi_max_test, max_indices = logit_mi_max(torch.Tensor(X_test), test_bags).detach().numpy()
#   print("ROC AUC: {}".format(get_roc_auc(test_labels, y_pred_logit_mi_max_test)))
#   print("PR AUC: {}".format(get_pr_auc(test_labels, y_pred_logit_mi_max_test)))
#   print("Accuracy Score: {}".format(accuracy_score(test_labels, y_pred_logit_mi_max_test >= 0.5)))
#   print("Balanced Accuracy Score: {}".format(balanced_accuracy_score(test_labels, y_pred_logit_mi_max_test >= 0.5)))

In [17]:
# logit_mi_mean = LogisticRegressionMI(64, mode='mean')
# criterion = BCELoss()
# optimizer = LBFGS(logit_mi_mean.parameters(), lr=0.001, max_iter=1000)
# logit_mi_mean.train()


# def closure():
#     optimizer.zero_grad()
#     output = logit_mi_mean(torch.Tensor(X_train), train_bags)
#     loss = criterion(output, torch.Tensor(train_labels))
#     loss.backward()
#     return loss

# optimizer.step(closure) 

# logit_mi_mean.eval()
# with torch.no_grad():
#   y_pred_logit_mi_mean_test = logit_mi_mean(torch.Tensor(X_test), test_bags).detach().numpy()
#   print("ROC AUC: {}".format(get_roc_auc(test_labels, y_pred_logit_mi_mean_test)))
#   print("PR AUC: {}".format(get_pr_auc(test_labels, y_pred_logit_mi_mean_test)))
#   print("Accuracy Score: {}".format(accuracy_score(test_labels, y_pred_logit_mi_mean_test >= 0.5)))
#   print("Balanced Accuracy Score: {}".format(balanced_accuracy_score(test_labels, y_pred_logit_mi_mean_test >= 0.5)))

In [28]:
# with torch.no_grad():
#   y_pred_logit_mi_max_test, max_indices = logit_mi_max.get_max_indices(torch.Tensor(X_test), test_bags)
#   y_pred_logit_mi_max_test = y_pred_logit_mi_max_test.detach().cpu().numpy()
#   print("ROC AUC: {}".format(get_roc_auc(test_labels, y_pred_logit_mi_max_test)))
#   print("PR AUC: {}".format(get_pr_auc(test_labels, y_pred_logit_mi_max_test)))
#   print("Accuracy Score: {}".format(accuracy_score(test_labels, y_pred_logit_mi_max_test >= 0.5)))
#   print("Balanced Accuracy Score: {}".format(balanced_accuracy_score(test_labels, y_pred_logit_mi_max_test >= 0.5)))

In [27]:
# pos_preds = (y_pred_logit_mi_max_test >= 0.5)
# pos_max_indices = max_indices[pos_preds]
# indices = np.random.choice(len(pos_max_indices), 5)
# fig, axes = plt.subplots(1, 5, figsize=(10, 50))
# for idx, ax in zip(indices, axes):
#   ax.imshow(mnist_arr_test[pos_max_indices][idx].reshape(28, 28))

In [26]:
# neg_preds = (y_pred_logit_mi_max_test < 0.5)

# neg_max_indices = max_indices[neg_preds]
# indices = np.random.choice(len(neg_max_indices), 5)
# fig, axes = plt.subplots(1, 5, figsize=(10, 50))
# for idx, ax in zip(indices, axes):
#   ax.imshow(mnist_arr_test[neg_max_indices][idx].reshape(28, 28))
#   ax.set_title(y_test[neg_max_indices][idx])